In [ ]:
#讀取文本中的文字進行分析
import pandas as pd

df = pd.read_csv("CustomerServiceRecord_20231204.csv")
df

In [245]:
for i in range(0, len(df)):
    if str(df["業務"][i]) == "nan":
        df.loc[i,["業務"]] = df["業務"][i-1]
    if str(df["客名"][i]) == "nan":
        df.loc[i,["客名"]] = df["客名"][i-1]
    if str(df["台數"][i]) == "nan":
        df.loc[i,["台數"]] = df["台數"][i-1]
    if str(df["服務日期"][i]) == "nan":
        df.loc[i,["服務日期"]] = df["服務日期"][i-1]
    if str(df["區域"][i]) == "nan":
        df.loc[i,["區域"]] = df["區域"][i-1]
    if str(df["客代"][i]) == "nan":
        df.loc[i,["客代"]] = df["客代"][i-1]
    if str(df["區域"][i]) == "nan":
        df.loc[i,["區域"]] = df["區域"][i-1]
    if str(df["行業別"][i]) == "nan":
        df.loc[i,["行業別"]] = df["行業別"][i-1]

In [5]:
df["服務類別"].value_counts()

服務類別
問題處理           43720
帳務處理           14964
故障處理           11116
拜訪客戶            5390
客服中心            4573
有望連繫            3743
主動關懷            1922
訂購產品            1358
要求拜訪             860
報價追蹤             518
洽詢產品             510
客情連繫             339
主動維護             238
其他               226
DM CALL OUT      108
紅利積點              51
試用產品              41
請介紹客戶             33
議價                33
客戶抱怨              13
工程主動觀察             8
問題反映               4
抱怨處理               4
帳務查詢               3
已收帳款               2
嚴重抱怨               1
Name: count, dtype: int64

In [313]:
#情感分析
from bixin import predict
from snownlp import SnowNLP

def emotionAnalysis(dataFrame, classifyName, text):
    positive = 0
    neutral = 0
    negative = 0
    
    for i in text:
        score = predict(i)
        if score > 0.3:
            positive += 1
        elif score > -0.3:
            neutral += 1
        elif score <= -0.9:
            s = SnowNLP(i)
            if s.sentiments <= 0.1:
                dataFrame.loc[len(dataFrame.index)] = [str(idDict[classifyName]), classifyName, areaDict[classifyName], salesmanDict[classifyName], carNumberDict[classifyName], i, score]
                negative += 1
    return positive, neutral, negative, dataFrame

In [314]:
#分析以業務為主的情緒分析
salesman = {}

#分析以10台以上的客戶為主的情緒分析
customer = {}

searchList = ["問題處理", "帳務處理", "故障處理", "客情連繫", "客戶抱怨", "抱怨處理", "嚴重抱怨"]

idDict = {}
areaDict = {}
salesmanDict = {}
carNumberDict = {}

for i in range(0, len(df)):
    if df["服務類別"][i] not in searchList:
        continue

    #存入業務為主分析
    if df["業務"][i] in salesman:
        salesman[df["業務"][i]].append(str(df["洽詢內容"][i]))
    elif df["業務"][i] not in salesman:
        salesman[df["業務"][i]] = [str(df["洽詢內容"][i])]

    #存入客戶為主分析
    if df["台數"][i] < 10:
        continue
    if df["客名"][i] in customer:
        customer[df["客名"][i]].append(str(df["洽詢內容"][i]))
    else:
        customer[df["客名"][i]] = [str(df["洽詢內容"][i])]
        idDict[df["客名"][i]] = df["客代"][i]
        areaDict[df["客名"][i]] = df["區域"][i]
        salesmanDict[df["客名"][i]] = df["業務"][i]
        carNumberDict[df["客名"][i]] = df["台數"][i]
    

In [315]:
#建立情緒分析檔案
badEmotionText = pd.DataFrame({'customerId': [],'customer':[], 'area':[], 'salesman':[], 'carNumber': [], 'text': [], 'score': []})

"""
customerEmotion = pd.DataFrame({'salesman':[], 'positive': [], 'neutral': [], 'negative': []})
for i in salesman:
    positive, neutral, negative, badEmotionText = emotionAnalysis(badEmotionText, i, salesman[i])
    customerEmotion.loc[len(customerEmotion.index)] = [i, positive, neutral, negative]

customerEmotion.to_csv("customerEmotionBySaleman.csv", encoding='utf-8-sig')

customerEmotion = pd.DataFrame({'customerId': [],'customer':[], 'area':[], 'salesman':[], 'carNumber': [], 'positive': [], 'neutral': [], 'negative': []})
"""
for i in customer:
    positive, neutral, negative, badEmotionText = emotionAnalysis(badEmotionText, i, customer[i])
    """
    if negative >= 40:
        print(i)
        print(positive)
        print(neutral)
        print(negative)
    customerEmotion.loc[len(customerEmotion.index)] = [idDict[i], i, areaDict[i], salesmanDict[i], carNumberDict[i], positive, neutral, negative]
    """
    
#customerEmotion.to_csv("customerEmotionByCustomer.csv", encoding='utf-8-sig')   
badEmotionText.to_csv("badEmotionTextByCustomer.csv", encoding='utf-8-sig')

In [292]:
#時間序列分析
#分析客戶不滿意度是否有提升


from bixin import predict
from snownlp import SnowNLP

def textEmotion(text):
    score = predict(text)
    if score > 0.3:
        return 1, 0, 0
    elif score > -0.3:
        return 0, 1, 0
    elif score <= -0.9:
        s = SnowNLP(text)
        if s.sentiments <= 0.1:
            return 0, 0, 1
        
    return 0, 1, 0

In [303]:

#月份的正向與負面次數
customer = {}

searchList = ["問題處理", "帳務處理", "故障處理", "客情連繫", "客戶抱怨", "抱怨處理", "嚴重抱怨"]

emotionByDate = pd.DataFrame({'date': [],'area':[], 'salesman':[], 'positive': [], 'neutral': [], 'negative': []})

for i in range(0, len(df)):
    if df["服務類別"][i] not in searchList and df["台數"][i] < 10:
        continue
    
    #存入時間為主分析    
    positive, neutral, negative = textEmotion(str(df["洽詢內容"][i]))
    emotionByDate.loc[len(emotionByDate.index)] = [df["服務日期"][i], df["區域"][i], df["業務"][i], positive, neutral, negative]


emotionByDate.to_csv("emotionByDate.csv", encoding='utf-8-sig')

In [ ]:
#業務擅長分析
salesmanDF = pd.DataFrame({'salesman': [],'industry':[], 'carNumber': []})

for i in range(0, len(df)):
    print(df["業務"][i])
    salesmanDF.loc[len(salesmanDF.index)] = [df["業務"][i], df["行業別"][i], df["台數"][i]]

salesmanDF.to_csv("salesmanDF.csv", encoding='utf-8-sig')

In [351]:
#關鍵字分析
#encoding=utf-8
from __future__ import unicode_literals
import jieba
import jieba.posseg
import jieba.analyse

In [352]:
# 載入詞庫
jieba.set_dictionary("dict.txt.big")
jieba.load_userdict("dict.txt.big")

# 加入字詞
jieba.add_word("Go168", tag="EUP")
jieba.add_word("大餅", tag="EUP")
jieba.add_word("DVR", tag="EUP")
jieba.add_word("油資", tag="EUP")
jieba.add_word("影像", tag="EUP")
jieba.add_word("3G", tag="EUP")
jieba.add_word("4G", tag="EUP")
jieba.add_word("5G", tag="EUP")
jieba.add_word("APP", tag="EUP")
jieba.add_word("環保大餅", tag="EUP")
jieba.add_word("中華電信", tag="EUP")
jieba.add_word("數位大餅", tag="EUP")
jieba.add_word("酒測器", tag="EUP")
jieba.add_word("ADAS", tag="EUP")
jieba.add_word("DMS", tag="EUP")
jieba.add_word("3轉4", tag="EUP")
jieba.add_word("雲市集", tag="EUP")
jieba.add_word("衛星犬", tag="EUP")

jieba.del_word("市集")
jieba.del_word("雲市")
jieba.del_word("電")
jieba.del_word("酒測")
jieba.del_word("衛星犬")
jieba.del_word("GPS")
jieba.del_word("補助")
jieba.del_word("車機")
jieba.del_word("費用")
jieba.del_word("系統")
jieba.del_word("車輛")
jieba.del_word("報價")
jieba.del_word("客戶")
jieba.del_word("我司")
jieba.del_word("時間")
jieba.del_word("價格")

Building prefix dict from /home/jovyan/work/CRM_Customer_Command/dict.txt.big ...
Loading model from cache /tmp/jieba.u4eac63a45549bf4a0d6bc2e5e14c909e.cache
Loading model cost 6.279 seconds.
Prefix dict has been built successfully.


In [378]:
#以月分做區分
from datetime import datetime

searchList = ["拜訪客戶", "有望連繫", "要求拜訪", "洽詢產品", "試用產品"]
competitor = [] # secret data need to remove
contentCustomer = pd.DataFrame({'customerId': [],'customer':[], 'text': []})

textJune = ""
textJuly = ""
textAug = ""
textSep = ""
textOct = ""
textNov = ""

for i in range(0, len(df)):
    if df["服務類別"][i] in searchList:
        contentDate = datetime.strptime(str(df["服務日期"][i]), '%Y/%m/%d').date()
        if contentDate < datetime.strptime("2023/07/01", '%Y/%m/%d').date():
            textJune += str(df["洽詢內容"][i]) + "。"
        elif contentDate < datetime.strptime("2023/08/01", '%Y/%m/%d').date():
            textJuly += str(df["洽詢內容"][i]) + "。"
        elif contentDate < datetime.strptime("2023/09/01", '%Y/%m/%d').date():
            textAug += str(df["洽詢內容"][i]) + "。"
        elif contentDate < datetime.strptime("2023/10/01", '%Y/%m/%d').date():
            textSep += str(df["洽詢內容"][i]) + "。"
        elif contentDate < datetime.strptime("2023/11/01", '%Y/%m/%d').date():
            textOct += str(df["洽詢內容"][i]) + "。"
        elif contentDate < datetime.strptime("2023/12/04", '%Y/%m/%d').date():
            textNov += str(df["洽詢內容"][i]) + "。"

        contentCustomer.loc[len(contentCustomer.index)] = [df["客代"][i], df["客名"][i], str(df["洽詢內容"][i])]

contentCustomer.to_csv("contentCustomer.csv", encoding='utf-8-sig')

In [374]:
from wordcloud import WordCloud
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

mask = np.array(Image.open('Icon.png')) #文字雲形狀
ignoreText = ["老闆", "問題", "聯繫", "老闆娘", "電話", "業務", "衛星犬", "報價單", "大哥", "方案", "資料", "理事長", "現場", "下單", "聯絡", "時候", "辦法", "車子", "科長", "課長", "襄理", "廠商", "技師",
              "發票", "月租", "狀況", "經理", "功能", "公司", "部分"]
flagList = ['n', 'ns', 'nt', 'nz', 'EUP']



def getKeyWord(textMon, label, csvDownload):
    keyWordText = ""
    top=0
    tag = jieba.analyse.extract_tags(textMon, topK=150, withWeight=True)
    for i in tag:
        #只取Top ?
        if top == 20:
            break
            
        words = jieba.posseg.cut(i[0])
        for word, flag in words:
            if word in ignoreText or len(word) == 1:
                continue
            if flag in flagList:
                top += 1

                #輸出檔案
                csvDownload.loc[len(csvDownload.index)] = [label, word, i[1]]
                
                #將文字比重化為次數
                textNum = int(i[1]*100)
                if textNum <= 0:
                    textNum = 1
                for i in range(0, textNum):
                    keyWordText += word + " "
                    
    return keyWordText, csvDownload

def drawTextCloud(cloudText, fileName):
    my_wordcloud = WordCloud(background_color='white', mask=mask, font_path="NotoSansTC-VariableFont_wght.ttf", collocations=False).generate(cloudText)
    my_wordcloud.to_file(fileName + '.png')
    plt.imshow(my_wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
monthCsvDownload = pd.DataFrame({'label': [],'keyword':[], 'weight':[]})

text6, monthCsvDownload = getKeyWord(textJune, "June", monthCsvDownload)
text7, monthCsvDownload = getKeyWord(textJuly, "July", monthCsvDownload)
text8, monthCsvDownload = getKeyWord(textAug, "Aug", monthCsvDownload)
text9, monthCsvDownload = getKeyWord(textSep, "Sep", monthCsvDownload)
text10, monthCsvDownload = getKeyWord(textOct, "Oct", monthCsvDownload)
text11, monthCsvDownload = getKeyWord(textNov, "Nov", monthCsvDownload)
#monthCsvDownload.to_csv("monthKeywordList.csv", encoding='utf-8-sig')

"""
print("=============六月份文字雲============")
drawTextCloud(text6, "juneCloud")


print("=============七月份文字雲============")
drawTextCloud(text7, "julyCloud")

print("=============八月份文字雲============")
drawTextCloud(text8, "auguestCloud")

print("=============九月份文字雲============")
drawTextCloud(text9, "septemberCloud")

print("=============十月份文字雲============")
drawTextCloud(text10, "octoberCloud")

print("=============十一月份文字雲============")
drawTextCloud(text11, "novemberCloud")
"""

print("=============近半年文字雲============")
drawTextCloud(text6+text7+text8+text9+text10+text11, "juneToNovemberCloud")



In [ ]:
#輸出關鍵字
text = textJune+textJuly+textAug+textSep+textOct+textNov
seg_lig = jieba.posseg.cut(text)
content = []
for w, tag in seg_lig:
    if 'n' in tag:
        if w in content:
            continue
        content.append(w)

kewWordList = pd.DataFrame({'word':content})
kewWordList.to_csv("kewWordList.csv", encoding='utf-8-sig')

In [340]:
#以地區做區分
from datetime import datetime

searchList = ["拜訪客戶", "有望連繫", "要求拜訪", "洽詢產品", "試用產品"]
area = {}
salesman = {}
industry = {}

for i in range(0, len(df)):
    if df["服務類別"][i] in searchList:
        #存入地區為主的分析
        if df["區域"][i] in area:
            area[df["區域"][i]] += str(df["洽詢內容"][i]) + "。"
        elif df["區域"][i] not in area:
            area[df["區域"][i]] = str(df["洽詢內容"][i]) + "。"

        #存入業務為主的分析
        if df["業務"][i] in salesman:
            salesman[df["業務"][i]] += str(df["洽詢內容"][i]) + "。"
        elif df["業務"][i] not in salesman:
            salesman[df["業務"][i]] = str(df["洽詢內容"][i]) + "。"

        #存入行業別為主的分析
        if df["行業別"][i] in salesman:
            industry[df["行業別"][i]] += str(df["洽詢內容"][i]) + "。"
        elif df["行業別"][i] not in salesman:
            industry[df["行業別"][i]] = str(df["洽詢內容"][i]) + "。"

In [ ]:
text = ""
areaCsvDownload = pd.DataFrame({'label': [],'keyword':[], 'weight':[]})
for i in area:
    text, areaCsvDownload = getKeyWord(area[i], i, areaCsvDownload)
    
    print("================= " + str(i) +" 的文字雲====================")
    drawTextCloud(text, str(i) + "Cloud")
    
#areaCsvDownload.to_csv("areaKeywordList.csv", encoding='utf-8-sig')

In [ ]:
text = ""
salesCsvDownload = pd.DataFrame({'label': [],'keyword':[], 'weight':[]})
for i in salesman:
    text, salesCsvDownload = getKeyWord(salesman[i], i, salesCsvDownload)
    
    print("================= " + str(i) +" 的文字雲====================")
    drawTextCloud(text, str(i) + "Cloud")
    
#salesCsvDownload.to_csv("salesKeywordList.csv", encoding='utf-8-sig')

In [ ]:
text = ""
industryCsvDownload = pd.DataFrame({'label': [],'keyword':[], 'weight':[]})
for i in industry:
    try:
        text, industryCsvDownload = getKeyWord(industry[i], i, industryCsvDownload)
        
        print("================= " + str(i) +" 的文字雲====================")
        drawTextCloud(text, str(i) + "Cloud")
    except:
        print(str(i) + "無法解析")
        print(industry[i])
    
#industryCsvDownload.to_csv("industryKeywordList.csv", encoding='utf-8-sig')